In [0]:
dbutils.fs.mount(source='wasbs://2ndcontainer@projectstorageaccount2nd.blob.core.windows.net',
                 mount_point='/mnt/blobstorage',
                 extra_configs={'fs.azure.account.key.projectstorageaccount2nd.blob.core.windows.net':'o049AtwBTNfEgkZwAFl+MdpvyMlZtIRc9JyR8MX0Ry5DqgryWwyt7qMFrGnN/8SNlH+9M+4Ab6uT+AStr9PcLw=='})

True

In [0]:
dbutils.fs.ls('/mnt/blobstorage/raw-data/')

[FileInfo(path='dbfs:/mnt/blobstorage/raw-data/Athletes.csv', name='Athletes.csv', size=418492, modificationTime=1710362963000),
 FileInfo(path='dbfs:/mnt/blobstorage/raw-data/EntriesGender.csv', name='EntriesGender.csv', size=1123, modificationTime=1710362991000),
 FileInfo(path='dbfs:/mnt/blobstorage/raw-data/Medals.csv', name='Medals.csv', size=2414, modificationTime=1710363004000),
 FileInfo(path='dbfs:/mnt/blobstorage/raw-data/Teams.csv', name='Teams.csv', size=35270, modificationTime=1710363020000),
 FileInfo(path='dbfs:/mnt/blobstorage/raw-data/coaches.csv', name='coaches.csv', size=16889, modificationTime=1710362977000)]

In [0]:
%fs
ls "/mnt/blobstorage/"

path,name,size,modificationTime
dbfs:/mnt/blobstorage/raw-data/,raw-data/,0,0
dbfs:/mnt/blobstorage/transformed_data/,transformed_data/,0,0


In [0]:
spark

In [0]:
Athletes = spark.read.format("csv").option("header","true").load("/mnt/blobstorage/raw-data/Athletes.csv")
coaches = spark.read.format("csv").option("header","true").load("/mnt/blobstorage/raw-data/coaches.csv")
EntriesGender = spark.read.format("csv").option("header","true").load("/mnt/blobstorage/raw-data/EntriesGender.csv")
Medals = spark.read.format("csv").option("header","true").load("/mnt/blobstorage/raw-data/Medals.csv")
Teams = spark.read.format("csv").option("header","true").load("/mnt/blobstorage/raw-data/Teams.csv")

In [0]:
Athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
EntriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
EntriesGender = EntriesGender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
Teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = Medals.orderBy("Gold", ascending=False).select("Team_Country","Gold").show()
     

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|                Cuba|   7|
|         New Zealand|   7|
|              Canada|   7|
|              Brazil|   7|
|   Republic of Korea|   6|
|             Hungary|   6|
|              Poland|   4|
|             Jamaica|   4|
|               Kenya|   4|
|      Czech Republic|   4|
|              Norway|   4|
|United States of ...|  39|
|People's Republic...|  38|
|              Sweden|   3|
|               Spain|   3|
|             Denmark|   3|
|         Switzerland|   3|
|             Croatia|   3|
|              Serbia|   3|
|Islamic Republic ...|   3|
+--------------------+----+
only showing top 20 rows



In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = EntriesGender.withColumn(
    'Avg_Female', EntriesGender['Female'] / EntriesGender['Total']
).withColumn(
    'Avg_Male', EntriesGender['Male'] / EntriesGender['Total']
)
average_entries_by_gender.show()
     

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
Athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/blobstorage/transformed_data/athletes")
     



In [0]:
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/blobstorage/transformed_data/coaches")
EntriesGender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/blobstorage/transformed_data/entriesgender")
Medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/blobstorage/transformed_data/medals")


In [0]:
Teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/blobstorage/transformed_data/teams")